In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
import random

Сгенерируем произвольную выборку:

In [2]:
random.seed(7)
X = np.random.randint(0,2,20).reshape(5,4) #randint(0,2,20).reshape(5,4) or randint(0,2,(5,4)) --> random((5,4),20) 
Y = np.random.randint(0,2,5)
X,Y

(array([[0, 1, 1, 1],
        [0, 1, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 1],
        [0, 1, 0, 1]]),
 array([1, 0, 1, 1, 0]))

In [3]:
params =  np.random.random((1,5)).T  # коэф. 4  и 1 порог

params

array([[0.90288466],
       [0.14493946],
       [0.81618397],
       [0.15215859],
       [0.64886741]])

Создаем функцию нахождения логита(сигмоид от параметра х). Логит находится один для одного вектора х_i, можно сказать эквивалента вероятности, так как результат всегда находится от 0 до 1.

In [4]:
def logit(x, alpha):
    return (1/(1+np.exp( -x@alpha[:-1]   + alpha[-1] ) ) )

logit(X,params).T, logit(X,params).T > .5 , Y.T == 1

(array([[0.61406092, 0.41295342, 0.37661803, 0.60017056, 0.41295342]]),
 array([[ True, False, False,  True, False]]),
 array([ True, False,  True,  True, False]))

Функция потерь, которая принимает на вход результат логита и выдает на сколько результат отклоняется от настоящего значения.  

Результат один для всех векторов x_i

In [34]:
def loss_function(x, alpha, y_):
    a = logit(x,alpha)
    l= -1. *( np.sum(  1.  *np.log( a[y_.T!=0]+1e-6 )   ) + np.sum(  np.log( 1. - a[y_.T==0] )  )   ) / len(y_)
    return l


Нужно оптимизировать параметры. Используем метод градиентного спуска: 

Будем искать производную функции потерь . Через **sympy**

In [6]:
import sympy 
from sympy import *
x,y,a, w0 = symbols( 'x y a w0')

In [7]:
expression = 1 / (1+ exp(-a*x + w0  ))
expression

1/(exp(-a*x + w0) + 1)

In [37]:
expression_loss_f = y*log(expression) + (1-y)*log(1-expression)
expression_loss_f

y*log(1/(exp(-a*x + w0) + 1)) + (1 - y)*log(1 - 1/(exp(-a*x + w0) + 1))

In [38]:
expression_loss_f_dif_a = diff(expression_loss_f,a)
expression_loss_f_dif_w0 = diff(expression_loss_f,w0)
expression_loss_f_dif_a

x*y*exp(-a*x + w0)/(exp(-a*x + w0) + 1) - x*(1 - y)*exp(-a*x + w0)/((1 - 1/(exp(-a*x + w0) + 1))*(exp(-a*x + w0) + 1)**2)

Теперь посчитаем для каждого x_i_j 

#### Буду считать градиент:

Считаем градиент для альфа (коэффициентов при х_i)

In [39]:
expression_loss_f_dif_a

x*y*exp(-a*x + w0)/(exp(-a*x + w0) + 1) - x*(1 - y)*exp(-a*x + w0)/((1 - 1/(exp(-a*x + w0) + 1))*(exp(-a*x + w0) + 1)**2)

In [11]:
grad_a = lambdify([y,x,a,w0] ,expression_loss_f_dif_a, 'numpy')

In [12]:
grad_a(Y[0],X[0],params[:-1].T,params[-1])

array([[0.        , 0.62338197, 0.45826817, 0.62168558]])

In [13]:
# коэфф для альфа [коэффициентов при х_i]
grad_a_sum_array = [0 , 0 , 0 , 0]

for  i in range( X.shape[0] ):
    l = grad_a(Y[i],X[i],params[:-1].T,params[-1]) 
    print( l )
    grad_a_sum_array += l
grad_a_sum_array = grad_a_sum_array / X.shape[0]
grad_a_sum_array

[[0.         0.62338197 0.45826817 0.62168558]]
[[ 0.         -0.37661803  0.         -0.37831442]]
[[0.         0.62338197 0.         0.        ]]
[[0.43683497 0.         0.         0.62168558]]
[[ 0.         -0.37661803  0.         -0.37831442]]


array([[0.08736699, 0.09870558, 0.09165363, 0.09734846]])

In [14]:
# упакуем в функцию grad_a_calculate
def grad_a_calculate(features, category, alpha):
    grad_a_sum_array = [0 , 0 , 0 , 0]
    
    for  i in range( features.shape[0] ):
        l = grad_a(category[i],features[i],alpha[:-1].T,alpha[-1]) 
        # print( l )
        grad_a_sum_array += l
    grad_a_sum_array = grad_a_sum_array / features.shape[0]
    return grad_a_sum_array

In [15]:
grad_a_calculate(X,Y,params)

array([[0.08736699, 0.09870558, 0.09165363, 0.09734846]])

Считаем градиент bias - как бы отступа для h(x_i) 

In [16]:
expression_loss_f_dif_w0

-y*exp(-a*x + w0)/(exp(-a*x + w0) + 1) + (1 - y)*exp(-a*x + w0)/((1 - 1/(exp(-a*x + w0) + 1))*(exp(-a*x + w0) + 1)**2)

In [17]:
grad_w0 = lambdify([y,x,a,w0] ,expression_loss_f_dif_w0, 'numpy')

In [18]:
grad_w0_sum_array = [0]

for  i in range( X.shape[0] ):
    l = grad_w0(Y[i],0,0,params[-1]) # x и alpha НЕ зависят от bias, то есть это получается как бы нулевой Х0=0 и alpha0=0 
    #print( l )
    grad_w0_sum_array += l
grad_w0_sum_array = grad_w0_sum_array / X.shape[0]
grad_w0_sum_array

array([-0.25675519])

In [19]:
# упакуем в функцию grad_w0_calculate
def grad_w0_calculate(features, category, alpha):
    grad_w0_sum_array = [0]

    for  i in range( features.shape[0] ):
        l = grad_w0(category[i],0,0,alpha[-1]) # x и alpha НЕ зависят от bias, то есть это получается как бы нулевой Х0=0 и alpha0=0 
        #print( l )
        grad_w0_sum_array += l
    grad_w0_sum_array = grad_w0_sum_array / features.shape[0]
    return grad_w0_sum_array

In [20]:
grad_w0_calculate(X,Y,params)

array([-0.25675519])

Запускаем градиентный спуск:

In [21]:
loss_function(X, params, Y)

1.6109531112540219

In [22]:
# Уменьшим дробную часть в выводе нампи
np.set_printoptions(suppress=True, formatter={ 'float_kind': '{:.3f}'.format  })

In [23]:
params =  np.random.random((1,5)).T 
learning_rate = 0.01
print('Loss funct \t|\t bias \t\t|\t alpha  ')
print('-'*85)

for i in range(1000):
    params[-1] -= learning_rate * grad_w0_calculate(X,Y,params)
    params[:-1] -= learning_rate * grad_a_calculate(X,Y,params).T
    l = loss_function(X, params, Y)
    print(f'{l :.3f}\t \t|\t {params[-1] } \t|\t {params[:-1].T }  ')
    if np.abs(l) < .1 : break

Loss funct 	|	 bias 		|	 alpha  
-------------------------------------------------------------------------------------
1.843	 	|	 [0.916] 	|	 [[0.549 0.227 0.766 0.213]]  
1.847	 	|	 [0.920] 	|	 [[0.548 0.226 0.765 0.212]]  
1.851	 	|	 [0.923] 	|	 [[0.547 0.225 0.764 0.211]]  
1.854	 	|	 [0.926] 	|	 [[0.545 0.223 0.762 0.209]]  
1.858	 	|	 [0.929] 	|	 [[0.544 0.222 0.761 0.208]]  
1.862	 	|	 [0.932] 	|	 [[0.543 0.220 0.760 0.206]]  
1.866	 	|	 [0.935] 	|	 [[0.542 0.219 0.759 0.205]]  
1.869	 	|	 [0.939] 	|	 [[0.541 0.218 0.758 0.204]]  
1.873	 	|	 [0.942] 	|	 [[0.539 0.216 0.757 0.202]]  
1.877	 	|	 [0.945] 	|	 [[0.538 0.215 0.756 0.201]]  
1.881	 	|	 [0.948] 	|	 [[0.537 0.213 0.755 0.199]]  
1.885	 	|	 [0.951] 	|	 [[0.536 0.212 0.754 0.198]]  
1.889	 	|	 [0.955] 	|	 [[0.535 0.211 0.753 0.196]]  
1.893	 	|	 [0.958] 	|	 [[0.533 0.209 0.752 0.195]]  
1.897	 	|	 [0.961] 	|	 [[0.532 0.208 0.750 0.194]]  
1.901	 	|	 [0.964] 	|	 [[0.531 0.206 0.749 0.192]]  
1.905	 	|	 [0.968] 	|	 [[0.530 0.

3.776	 	|	 [1.777] 	|	 [[0.232 -0.273 0.470 -0.292]]  
3.790	 	|	 [1.782] 	|	 [[0.230 -0.276 0.468 -0.295]]  
3.805	 	|	 [1.786] 	|	 [[0.229 -0.279 0.467 -0.298]]  
3.819	 	|	 [1.791] 	|	 [[0.227 -0.282 0.465 -0.301]]  
3.834	 	|	 [1.795] 	|	 [[0.225 -0.285 0.464 -0.304]]  
3.849	 	|	 [1.800] 	|	 [[0.224 -0.288 0.462 -0.307]]  
3.864	 	|	 [1.805] 	|	 [[0.222 -0.292 0.461 -0.310]]  
3.879	 	|	 [1.809] 	|	 [[0.220 -0.295 0.459 -0.313]]  
3.893	 	|	 [1.814] 	|	 [[0.219 -0.298 0.457 -0.317]]  
3.908	 	|	 [1.818] 	|	 [[0.217 -0.301 0.456 -0.320]]  
3.923	 	|	 [1.823] 	|	 [[0.215 -0.304 0.454 -0.323]]  
3.938	 	|	 [1.828] 	|	 [[0.214 -0.307 0.453 -0.326]]  
3.954	 	|	 [1.832] 	|	 [[0.212 -0.310 0.451 -0.329]]  
3.969	 	|	 [1.837] 	|	 [[0.210 -0.314 0.449 -0.332]]  
3.984	 	|	 [1.841] 	|	 [[0.209 -0.317 0.448 -0.336]]  
3.999	 	|	 [1.846] 	|	 [[0.207 -0.320 0.446 -0.339]]  
4.014	 	|	 [1.851] 	|	 [[0.205 -0.323 0.445 -0.342]]  
4.030	 	|	 [1.855] 	|	 [[0.204 -0.326 0.443 -0.345]]  
4.045	 	|	

8.435	 	|	 [3.046] 	|	 [[-0.218 -1.164 0.031 -1.184]]  
8.457	 	|	 [3.052] 	|	 [[-0.220 -1.168 0.029 -1.188]]  
8.478	 	|	 [3.057] 	|	 [[-0.222 -1.172 0.027 -1.192]]  
8.499	 	|	 [3.063] 	|	 [[-0.224 -1.176 0.025 -1.196]]  
8.521	 	|	 [3.068] 	|	 [[-0.226 -1.179 0.023 -1.200]]  
8.542	 	|	 [3.074] 	|	 [[-0.228 -1.183 0.021 -1.204]]  
8.563	 	|	 [3.079] 	|	 [[-0.230 -1.187 0.019 -1.208]]  
8.585	 	|	 [3.085] 	|	 [[-0.232 -1.191 0.018 -1.212]]  
8.606	 	|	 [3.090] 	|	 [[-0.233 -1.195 0.016 -1.216]]  
8.628	 	|	 [3.096] 	|	 [[-0.235 -1.199 0.014 -1.220]]  
8.649	 	|	 [3.102] 	|	 [[-0.237 -1.203 0.012 -1.223]]  
8.671	 	|	 [3.107] 	|	 [[-0.239 -1.207 0.010 -1.227]]  
8.692	 	|	 [3.113] 	|	 [[-0.241 -1.211 0.008 -1.231]]  
8.714	 	|	 [3.118] 	|	 [[-0.243 -1.214 0.006 -1.235]]  
8.735	 	|	 [3.124] 	|	 [[-0.245 -1.218 0.004 -1.239]]  
8.757	 	|	 [3.129] 	|	 [[-0.247 -1.222 0.002 -1.243]]  
8.778	 	|	 [3.135] 	|	 [[-0.249 -1.226 0.000 -1.247]]  
8.800	 	|	 [3.141] 	|	 [[-0.251 -1.230 -0.002 -1

14.070	 	|	 [4.514] 	|	 [[-0.720 -2.173 -0.468 -2.194]]  
14.092	 	|	 [4.520] 	|	 [[-0.722 -2.177 -0.470 -2.198]]  
14.115	 	|	 [4.525] 	|	 [[-0.724 -2.181 -0.472 -2.201]]  
14.137	 	|	 [4.531] 	|	 [[-0.726 -2.185 -0.474 -2.205]]  
14.159	 	|	 [4.537] 	|	 [[-0.728 -2.189 -0.476 -2.209]]  
14.182	 	|	 [4.543] 	|	 [[-0.730 -2.193 -0.478 -2.213]]  
14.204	 	|	 [4.549] 	|	 [[-0.732 -2.197 -0.480 -2.217]]  
14.227	 	|	 [4.555] 	|	 [[-0.734 -2.201 -0.482 -2.221]]  
14.249	 	|	 [4.561] 	|	 [[-0.736 -2.205 -0.484 -2.225]]  
14.272	 	|	 [4.567] 	|	 [[-0.738 -2.209 -0.486 -2.229]]  
14.294	 	|	 [4.573] 	|	 [[-0.740 -2.213 -0.488 -2.233]]  
14.316	 	|	 [4.578] 	|	 [[-0.742 -2.217 -0.490 -2.237]]  
14.339	 	|	 [4.584] 	|	 [[-0.743 -2.221 -0.492 -2.241]]  
14.361	 	|	 [4.590] 	|	 [[-0.745 -2.225 -0.494 -2.245]]  
14.384	 	|	 [4.596] 	|	 [[-0.747 -2.229 -0.496 -2.249]]  
14.406	 	|	 [4.602] 	|	 [[-0.749 -2.233 -0.498 -2.253]]  
14.428	 	|	 [4.608] 	|	 [[-0.751 -2.237 -0.500 -2.257]]  
14.451	 	|	 [4

17.244	 	|	 [5.361] 	|	 [[-1.005 -2.744 -0.753 -2.765]]  
17.266	 	|	 [5.367] 	|	 [[-1.007 -2.748 -0.755 -2.769]]  
17.288	 	|	 [5.373] 	|	 [[-1.009 -2.752 -0.757 -2.773]]  
17.309	 	|	 [5.379] 	|	 [[-1.011 -2.756 -0.759 -2.777]]  
17.331	 	|	 [5.385] 	|	 [[-1.013 -2.760 -0.761 -2.781]]  
17.353	 	|	 [5.391] 	|	 [[-1.015 -2.764 -0.763 -2.785]]  
17.374	 	|	 [5.397] 	|	 [[-1.017 -2.768 -0.765 -2.789]]  
17.396	 	|	 [5.403] 	|	 [[-1.019 -2.772 -0.767 -2.793]]  
17.417	 	|	 [5.409] 	|	 [[-1.021 -2.776 -0.769 -2.797]]  
17.439	 	|	 [5.415] 	|	 [[-1.023 -2.780 -0.771 -2.801]]  
17.460	 	|	 [5.421] 	|	 [[-1.025 -2.784 -0.773 -2.805]]  
17.482	 	|	 [5.427] 	|	 [[-1.027 -2.788 -0.775 -2.809]]  
17.503	 	|	 [5.433] 	|	 [[-1.029 -2.792 -0.777 -2.813]]  
17.525	 	|	 [5.439] 	|	 [[-1.031 -2.796 -0.779 -2.817]]  
17.546	 	|	 [5.444] 	|	 [[-1.033 -2.800 -0.781 -2.821]]  
17.568	 	|	 [5.450] 	|	 [[-1.035 -2.804 -0.783 -2.825]]  
17.589	 	|	 [5.456] 	|	 [[-1.037 -2.808 -0.785 -2.829]]  
17.610	 	|	 [5

In [24]:
logit(X,params).T,   (logit(X,params).T > .5).astype(int)   ,   Y.T

(array([[0.000, 0.000, 0.000, 0.000, 0.000]]),
 array([[0, 0, 0, 0, 0]]),
 array([1, 0, 1, 1, 0]))

Выполним задачу для цветочков ириса:

In [25]:
from sklearn.datasets import load_iris

iris = load_iris()

# Зкгрузим вектора и разметку
X = iris.data
Y = iris.target

In [26]:
# сгенерим рандомные коэффициенты и биас
params =  np.random.random((1,5)).T  # коэф. 4  и 1 порог

params

array([[0.226],
       [0.810],
       [0.803],
       [0.832],
       [0.686]])

In [27]:
import copy

Y_ = copy.copy(Y) 
np.place(Y_, Y_ != 0, 1)
Y , Y_

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [35]:
# сгенерим рандомные коэффициенты и биас
params =  np.random.random((1,5)).T  # коэф. 4  и 1 порог

learning_rate = 0.01
print('Loss funct \t|\t bias \t\t|\t alpha  ')
print('-'*85)

for i in range(100000):
    params[-1] -= learning_rate * grad_w0_calculate(X,Y_,params)
    params[:-1] -= learning_rate * grad_a_calculate(X,Y_,params).T
    l = loss_function(X, params, Y_)
    print(f'{l :.3f}\t \t|\t {params[-1] } \t|\t {params[:-1].T }  ')
    if np.abs(l) < 1. : break

Loss funct 	|	 bias 		|	 alpha  
-------------------------------------------------------------------------------------
1.804	 	|	 [0.727] 	|	 [[0.561 0.914 0.104 0.134]]  
1.824	 	|	 [0.731] 	|	 [[0.574 0.922 0.088 0.127]]  
1.844	 	|	 [0.734] 	|	 [[0.586 0.930 0.071 0.121]]  
1.864	 	|	 [0.737] 	|	 [[0.599 0.938 0.053 0.114]]  
1.885	 	|	 [0.741] 	|	 [[0.612 0.946 0.035 0.107]]  
1.905	 	|	 [0.744] 	|	 [[0.625 0.954 0.016 0.100]]  
1.925	 	|	 [0.748] 	|	 [[0.639 0.963 -0.004 0.093]]  
1.946	 	|	 [0.751] 	|	 [[0.653 0.971 -0.025 0.086]]  
1.967	 	|	 [0.755] 	|	 [[0.667 0.979 -0.047 0.079]]  
1.987	 	|	 [0.758] 	|	 [[0.681 0.988 -0.069 0.072]]  
2.008	 	|	 [0.762] 	|	 [[0.695 0.996 -0.092 0.065]]  
2.028	 	|	 [0.765] 	|	 [[0.710 1.005 -0.116 0.058]]  
2.049	 	|	 [0.769] 	|	 [[0.724 1.013 -0.141 0.051]]  
2.069	 	|	 [0.772] 	|	 [[0.739 1.022 -0.166 0.044]]  
2.089	 	|	 [0.776] 	|	 [[0.754 1.031 -0.192 0.037]]  
2.110	 	|	 [0.779] 	|	 [[0.769 1.040 -0.219 0.030]]  
2.130	 	|	 [0.783] 	|	 

5.991	 	|	 [1.334] 	|	 [[3.035 2.498 -4.635 -1.260]]  
6.034	 	|	 [1.338] 	|	 [[3.052 2.509 -4.668 -1.271]]  
6.077	 	|	 [1.343] 	|	 [[3.069 2.521 -4.701 -1.282]]  
6.120	 	|	 [1.347] 	|	 [[3.085 2.532 -4.734 -1.292]]  
6.164	 	|	 [1.352] 	|	 [[3.102 2.543 -4.766 -1.303]]  
6.208	 	|	 [1.357] 	|	 [[3.119 2.555 -4.799 -1.314]]  
6.251	 	|	 [1.361] 	|	 [[3.135 2.566 -4.832 -1.325]]  
6.295	 	|	 [1.366] 	|	 [[3.152 2.577 -4.864 -1.335]]  
6.339	 	|	 [1.370] 	|	 [[3.169 2.589 -4.897 -1.346]]  
6.384	 	|	 [1.375] 	|	 [[3.185 2.600 -4.930 -1.357]]  
6.428	 	|	 [1.380] 	|	 [[3.202 2.612 -4.962 -1.368]]  
6.473	 	|	 [1.384] 	|	 [[3.219 2.623 -4.995 -1.378]]  
6.517	 	|	 [1.389] 	|	 [[3.235 2.634 -5.028 -1.389]]  
6.562	 	|	 [1.394] 	|	 [[3.252 2.646 -5.061 -1.400]]  
6.607	 	|	 [1.398] 	|	 [[3.269 2.657 -5.093 -1.411]]  
6.652	 	|	 [1.403] 	|	 [[3.285 2.668 -5.126 -1.422]]  
6.697	 	|	 [1.408] 	|	 [[3.302 2.680 -5.159 -1.432]]  
6.742	 	|	 [1.412] 	|	 [[3.319 2.691 -5.191 -1.443]]  
6.788	 	|	

nan	 	|	 [2.155] 	|	 [[nan 4.323 -9.868 -3.020]]  
nan	 	|	 [2.160] 	|	 [[nan 4.334 -9.901 -3.031]]  
nan	 	|	 [2.166] 	|	 [[nan 4.346 -9.934 -3.042]]  
nan	 	|	 [2.172] 	|	 [[nan 4.357 -9.967 -3.053]]  
nan	 	|	 [2.177] 	|	 [[nan 4.369 -9.999 -3.064]]  
nan	 	|	 [2.183] 	|	 [[nan 4.380 -10.032 -3.075]]  
nan	 	|	 [2.189] 	|	 [[nan 4.392 -10.065 -3.086]]  
nan	 	|	 [2.194] 	|	 [[nan 4.403 -10.097 -3.098]]  
nan	 	|	 [2.200] 	|	 [[nan 4.414 -10.130 -3.109]]  
nan	 	|	 [2.206] 	|	 [[nan 4.426 -10.163 -3.120]]  
nan	 	|	 [2.211] 	|	 [[nan 4.437 -10.196 -3.131]]  
nan	 	|	 [2.217] 	|	 [[nan 4.449 -10.228 -3.142]]  
nan	 	|	 [2.223] 	|	 [[nan 4.460 -10.261 -3.153]]  
nan	 	|	 [2.228] 	|	 [[nan 4.472 -10.294 -3.164]]  
nan	 	|	 [2.234] 	|	 [[nan 4.483 -10.326 -3.175]]  
nan	 	|	 [2.240] 	|	 [[nan 4.494 -10.359 -3.187]]  
nan	 	|	 [2.245] 	|	 [[nan 4.506 -10.392 -3.198]]  
nan	 	|	 [2.251] 	|	 [[nan 4.517 -10.424 -3.209]]  
nan	 	|	 [2.257] 	|	 [[nan 4.529 -10.457 -3.220]]  
nan	 	|	 [2.263] 

nan	 	|	 [3.122] 	|	 [[nan 6.174 -15.167 -4.826]]  
nan	 	|	 [3.128] 	|	 [[nan 6.185 -15.200 -4.837]]  
nan	 	|	 [3.135] 	|	 [[nan 6.197 -15.232 -4.848]]  
nan	 	|	 [3.141] 	|	 [[nan 6.208 -15.265 -4.859]]  
nan	 	|	 [3.147] 	|	 [[nan 6.220 -15.298 -4.870]]  
nan	 	|	 [3.153] 	|	 [[nan 6.231 -15.330 -4.881]]  
nan	 	|	 [3.160] 	|	 [[nan 6.243 -15.363 -4.893]]  
nan	 	|	 [3.166] 	|	 [[nan 6.254 -15.396 -4.904]]  
nan	 	|	 [3.172] 	|	 [[nan 6.265 -15.429 -4.915]]  
nan	 	|	 [3.178] 	|	 [[nan 6.277 -15.461 -4.926]]  
nan	 	|	 [3.185] 	|	 [[nan 6.288 -15.494 -4.937]]  
nan	 	|	 [3.191] 	|	 [[nan 6.300 -15.527 -4.948]]  


KeyboardInterrupt: 

In [ ]:
loss_function(X, params, y)

In [ ]:
logit(X,params).T, logit(X,params).T < .5 , y_.T == 0